In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
np.random.seed(203)
from tqdm import tqdm
import datetime
from collections import Counter
import re

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb
from sklearn.metrics import precision_score , recall_score , f1_score

In [ ]:
data_train = pd.read_csv('/Users/s0c02nj/Desktop/Mortgage Modelling/CAX_MortgageModeling_Train.csv')
data_test = pd.read_csv('/Users/s0c02nj/Desktop/Mortgage Modelling/CAX_MortgageModeling_Test.csv')
data_sub = pd.read_csv('/Users/s0c02nj/Desktop/Mortgage Modelling/CAX_MortgageModeling_SubmissionFormat.csv')

In [ ]:
#data_train['FSA'].unique()

In [ ]:
(data_train['RESULT'].value_counts()[0])

In [ ]:
(data_train['RESULT'].value_counts()[1])

In [ ]:
cont_cols = ['PROPERTY VALUE','MORTGAGE PAYMENT','GDS','LTV','TDS','AMORTIZATION','MORTGAGE AMOUNT',
             'INCOME','CREDIT SCORE','RATE']

In [ ]:
#data_train['AMORTIZATION'].unique()

In [ ]:
#sns.distplot(np.log1p(data_train['RATE']))

In [ ]:
log_transformed = ['PROPERTY VALUE','MORTGAGE PAYMENT','LTV','INCOME','CREDIT SCORE']

In [ ]:
non_log_cols = ['GDS','LTV','TDS','AMORTIZATION','RATE']

In [ ]:
cat_cols = ['PAYMENT FREQUENCY','PROPERTY TYPE','FSA','TERM','AGE RANGE','GENDER','INCOME TYPE',
           'NAICS CODE','MORTGAGE PURPOSE']

Preprocessing

In [ ]:
data_train = data_train.drop(['Unique_ID','MORTGAGE NUMBER'],axis=1)
data_test =  data_test.drop(['Unique_ID','MORTGAGE NUMBER','RESULT'], axis=1)

In [ ]:
#Train data separating the response
y       = data_train['RESULT']
x_train = data_train.drop(['RESULT'],axis=1)


In [ ]:
#Test data and train data merged for pre-processing
x_comb = pd.concat([x_train,data_test],sort=False)

In [ ]:
cont_cols = ['PROPERTY VALUE','MORTGAGE PAYMENT','GDS','LTV','TDS','AMORTIZATION','MORTGAGE AMOUNT',
             'INCOME','CREDIT SCORE','RATE']

In [ ]:
cat_cols = ['PAYMENT FREQUENCY','PROPERTY TYPE','FSA','TERM','AGE RANGE','GENDER','INCOME TYPE',
           'NAICS CODE','MORTGAGE PURPOSE']

Label Encoding

In [ ]:
for col in tqdm(cat_cols):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col])

In [ ]:
#Label Encoding the y-var
le_y = LabelEncoder()

y_cat = le_y.fit_transform(y)

In [ ]:
{'FUNDED':0,'NOT FUNDED':1}

In [ ]:
for col in log_transformed:
    x_comb[col] = np.log1p(x_comb[col])

In [ ]:
for i,col in tqdm(enumerate(cont_cols)):
    counter = Counter(x_comb[col])
    x_comb[str(col)+'count'] = x_comb[col].apply(lambda x:counter[x])

In [ ]:
train_x = x_comb[0:45642]
test_x =  x_comb[45642:]

In [ ]:
x1_train, x1_val, y1_train, y1_val = train_test_split(train_x, y_cat,test_size=0.2,random_state=0,stratify=y)

In [ ]:
random_state = 42
np.random.seed(random_state)

lgb_params = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : 10,
    "num_leaves" : 20,
    "learning_rate" : 0.02,
    "bagging_freq": 5,
    "bagging_fraction" : 0.60,
    "feature_fraction" : 0.08,
    "min_data_in_leaf": 40,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    'n_esimators' : 500,
    'min_split_gain': .001,
    'reg_alpha': .01,
    'reg_lambda': .00001,
    "bagging_seed" : random_state,
    "verbosity" : 1,
    "seed": random_state
}

In [ ]:
trn_data = lgb.Dataset(x1_train, label= y1_train, categorical_feature= cat_cols)
val_data = lgb.Dataset(x1_val, label=  y1_val, categorical_feature=cat_cols)

In [ ]:
lgb_clf = lgb.train(lgb_params,
                    trn_data,
                    13000,
                    valid_sets = [trn_data, val_data],
                    early_stopping_rounds=3000,
                    verbose_eval = 1000
                    )

In [ ]:
train_x['MORTGAGE PURPOSE'].unique()

In [ ]:
data_train['MORTGAGE PURPOSE'].unique()

In [ ]:
pred = model1.predict(x_test_comb)

In [ ]:
#Pred
y_classes = pred.argmax(axis=-1)

In [ ]:
y_final = list(le_y.inverse_transform(y_classes))

In [ ]:
data_sub['amount_spent_per_room_night_scaled']=pred

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/Mahindra/Sub_cat7.csv',index=False)